모델을 불러오는 과정

In [1]:
import cv2
from ultralytics import YOLO
from ultralytics.engine.results import Results

# Load a pretrained YOLOv8n model
model = YOLO('yolov8x-seg.pt')

# Run inference on an image
results: Results = model('예제선반1.jpg')  # results list


image 1/1 l:\2024-1\SW\0429   \Arrangement_of_Jinn_Dolsoe\1.jpg: 480x640 4 bottles, 1 cup, 562.4ms
Speed: 1.0ms preprocess, 562.4ms inference, 477.0ms postprocess per image at shape (1, 3, 480, 640)


선반을 관리하는 클래스

In [10]:
class Shelf:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = min(x1, x2)
        self.y1 = min(y1, y2)
        self.x2 = max(x1, x2)
        self.y2 = max(y1, y2)
        self.width = self.x2 - self.x1
        self.height = self.y2 - self.y1

    def __str__(self):
        return f'x1: {self.x1}, y1: {self.y1}, x2: {self.x2}, y2: {self.y2}'
    
    def print(self):
        print(f"선반의 가로 길이: {self.width}, 선반의 세로 길이: {self.height}")

마우스 드래그로 사각형을 그리는 기능

In [11]:
# Global variables
drawing = False # true if mouse is pressed
ix, iy = -1, -1
img = None
temp_img = None
shelf_object = list()

# Mouse callback function
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, img, temp_img

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y
        temp_img = img.copy()

    elif event == cv2.EVENT_MOUSEMOVE:  
        if drawing is True:
            img = temp_img.copy()
            cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), 1)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        cv2.rectangle(img, (ix, iy), (x, y), (0, 255, 0), -1)
        shelf_object.append(Shelf(ix, iy, x, y))
        print(f"선반의 좌표: {shelf_object[-1]}")
        print(f"선반의 가로 길이: {shelf_object[-1].width}, 선반의 세로 길이: {shelf_object[-1].height}")

# Load an image
img = results[0].orig_img.copy()
cv2.namedWindow('image')
cv2.setMouseCallback('image', draw_rectangle)

while(1):
    cv2.imshow('image', img)
    if cv2.waitKey(1) & 0xFF == 27:  # exit if ESC is pressed
        break

cv2.destroyAllWindows()

선반의 좌표: x1: 477, y1: 263, x2: 1117, y2: 795
선반의 가로 길이: 640, 선반의 세로 길이: 532
선반의 좌표: x1: 167, y1: 348, x2: 359, y2: 575
선반의 가로 길이: 192, 선반의 세로 길이: 227


탐지된 객체를 관리하는 클래스

In [4]:
class YoloObject:
    def __init__(self, object):
        self.object = object

    def show(self):
        self.object.show()

객체를 하나씩 보여주고 가로 세로 길이를 출력하는 기능

In [5]:
image_path = '예제선반1.jpg'
image = cv2.imread(image_path)

for r in results[0]:
    for idx, box in enumerate(r.boxes.xyxy):
        x1, y1, x2, y2 = map(int, box[:4])  # 경계 상자 좌표 (정수로 변환)
        cropped_img = image[y1:y2, x1:x2]  # 경계 상자 좌표를 사용하여 이미지 자르기

        height, width = cropped_img.shape[:2]
        new_width = int(width)
        new_height = int(height)
        cropped_img = cv2.resize(cropped_img, (new_width, new_height))
            
        print(f'물건 {idx+1} 가로 길이: {new_width}, 세로 길이: {new_height}')  # 객체의 가로와 세로 길이 출력
        cv2.imshow(f'Detected Object {idx+1}', cropped_img)
        cv2.waitKey(0)

cv2.destroyAllWindows() # 이미지창 닫기

물건 1 가로 길이: 112, 세로 길이: 322
물건 1 가로 길이: 125, 세로 길이: 431
물건 1 가로 길이: 110, 세로 길이: 173
물건 1 가로 길이: 138, 세로 길이: 460
물건 1 가로 길이: 54, 세로 길이: 199
